In [40]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install gseapy pandas matplotlib seaborn
import gseapy as gp
import seaborn as sns
from gseapy.plot import gseaplot2
!pip install gseapy pandas matplotlib seaborn sqlalchemy

In [41]:
import pandas as pd
from sqlalchemy import create_engine

db_path = "P:/Projects/WFB_SIA_2024_Jaitovich_LongCOVID/Database/Long Covid Study DB.sqlite"
engine = create_engine(f'sqlite:///{db_path}')



In [42]:
biomolecules_df = pd.read_sql(
    "SELECT biomolecule_id, omics_id, keep FROM biomolecules WHERE omics_id = 2 AND keep = 1",
    engine
)

print("\nMerged table preview:", biomolecules_df, sep="\n")



Merged table preview:
      biomolecule_id  omics_id keep
0              38167         2    1
1              38168         2    1
2              38172         2    1
3              38173         2    1
4              38174         2    1
5              38176         2    1
6              38177         2    1
7              38179         2    1
8              38180         2    1
9              38181         2    1
10             38183         2    1
11             38186         2    1
12             38188         2    1
13             38189         2    1
14             38195         2    1
15             38199         2    1
16             38200         2    1
17             38201         2    1
18             38207         2    1
19             38208         2    1
20             38210         2    1
21             38212         2    1
22             38214         2    1
23             38217         2    1
24             38222         2    1
25             38223         2    1
26   

In [72]:
# Read from the database
pvalues = pd.read_sql(
    '''
    SELECT biomolecule_id, comparison, analysis_group, effect_size, p_value, q_value, formula
    FROM pvalues
    WHERE analysis_group = 1
    ''',
    engine
)

# Ensure formula column is float
pvalues['formula'] = pvalues['formula'].astype(float)

# Filter using numeric values
pvalues = pvalues[pvalues['formula'].isin([22.0, 23.0, 24.0, 25.0])]

# Check results
print("\nFiltered pvalues:")
print(pvalues.head())




Filtered pvalues:
        biomolecule_id comparison analysis_group  effect_size   p_value  \
248680               1        QoL              1    -0.000276  0.304654   
248681               3        QoL              1    -0.000548  0.069132   
248682               5        QoL              1    -0.000231  0.594754   
248683               7        QoL              1    -0.000496  0.200286   
248684              12        QoL              1    -0.000765  0.019310   

         q_value  formula  
248680  0.568483     22.0  
248681  0.252821     22.0  
248682  0.792623     22.0  
248683  0.454539     22.0  
248684  0.123258     22.0  


In [73]:
biomolecules_metadata = pd.read_sql(
    """
    SELECT biomolecule_id, metadata_type, metadata_value
    FROM biomolecules_metadata
    WHERE metadata_type IN ('Lipid_Category', 'Main_Class', 'Unsaturation_Level', 'NumFattyAcylCarbons')
    """,
    engine
)


In [74]:
metadata_wide = biomolecules_metadata.pivot_table(
    index='biomolecule_id',
    columns='metadata_type',
    values='metadata_value',
    aggfunc='first'  # Use first in case of duplicates
).reset_index()

print(metadata_wide.columns.tolist())

print("Shape of metadata_wide:", metadata_wide.shape)


['biomolecule_id', 'Lipid_Category', 'Main_Class', 'NumFattyAcylCarbons', 'Unsaturation_Level']
Shape of metadata_wide: (2658, 5)


In [75]:
metadata_wide['biomolecule_id'] = metadata_wide['biomolecule_id'].astype(str)
biomolecules_df['biomolecule_id'] = biomolecules_df['biomolecule_id'].astype(str)


metadata_joined = pd.merge(metadata_wide, biomolecules_df[['biomolecule_id', 'keep']], on='biomolecule_id', how='inner')

# Filter for only quantifiable values 
metadata_filtered = metadata_joined[metadata_joined['keep'] == '1'].copy()
metadata_filtered = metadata_filtered.drop(columns='keep')

print("\nFinal metadata table with keep==1:")
print(metadata_filtered.head())
print("Shape:", metadata_filtered.shape)


Final metadata table with keep==1:
  biomolecule_id Lipid_Category    Main_Class NumFattyAcylCarbons  \
0          38167    Fatty Acyls  Fatty esters                  10   
1          38168    Fatty Acyls  Fatty esters                  11   
2          38172    Fatty Acyls  Fatty esters                  12   
3          38173    Fatty Acyls  Fatty esters                  12   
4          38174    Fatty Acyls  Fatty esters                  13   

  Unsaturation_Level  
0           Very Low  
1           Very Low  
2           Very Low  
3           Very Low  
4           Very Low  
Shape: (1729, 5)


In [76]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print("Dimensions of metadata_filtered:", metadata_filtered.shape)

print(metadata_filtered.head())


Dimensions of metadata_filtered: (1729, 5)
  biomolecule_id Lipid_Category    Main_Class NumFattyAcylCarbons  \
0          38167    Fatty Acyls  Fatty esters                  10   
1          38168    Fatty Acyls  Fatty esters                  11   
2          38172    Fatty Acyls  Fatty esters                  12   
3          38173    Fatty Acyls  Fatty esters                  12   
4          38174    Fatty Acyls  Fatty esters                  13   

  Unsaturation_Level  
0           Very Low  
1           Very Low  
2           Very Low  
3           Very Low  
4           Very Low  


In [77]:

metadata_filtered['biomolecule_id'] = metadata_filtered['biomolecule_id'].astype(str)
pvalues['biomolecule_id'] = pvalues['biomolecule_id'].astype(str)

merged_metadata_pvalues = pvalues.merge(metadata_filtered, on='biomolecule_id', how='inner')

print("Merged metadata + pvalues shape:", merged_metadata_pvalues.shape)
print(merged_metadata_pvalues.head())


Merged metadata + pvalues shape: (6916, 11)
  biomolecule_id comparison analysis_group  effect_size   p_value   q_value  \
0          38167        QoL              1     0.000488  0.220141  0.478704   
1          38168        QoL              1     0.000352  0.154253  0.394402   
2          38172        QoL              1     0.000164  0.607183  0.800352   
3          38173        QoL              1     0.000140  0.663022  0.835220   
4          38174        QoL              1     0.001174  0.000278  0.009459   

   formula Lipid_Category    Main_Class NumFattyAcylCarbons Unsaturation_Level  
0     22.0    Fatty Acyls  Fatty esters                  10           Very Low  
1     22.0    Fatty Acyls  Fatty esters                  11           Very Low  
2     22.0    Fatty Acyls  Fatty esters                  12           Very Low  
3     22.0    Fatty Acyls  Fatty esters                  12           Very Low  
4     22.0    Fatty Acyls  Fatty esters                  13           Very L

In [78]:
cols_to_check = ['Lipid_Category', 'Main_Class', 'Unsaturation_Level', 'NumFattyAcylCarbons']

merged_metadata_pvalues[cols_to_check] = merged_metadata_pvalues[cols_to_check].replace(
    to_replace=[None, '', np.nan],
    value='Unknown'
)

print("\nCleaned metadata_wide preview (all unknowns standardized):", merged_metadata_pvalues.head(), sep="\n")



Cleaned metadata_wide preview (all unknowns standardized):
  biomolecule_id comparison analysis_group  effect_size   p_value   q_value  \
0          38167        QoL              1     0.000488  0.220141  0.478704   
1          38168        QoL              1     0.000352  0.154253  0.394402   
2          38172        QoL              1     0.000164  0.607183  0.800352   
3          38173        QoL              1     0.000140  0.663022  0.835220   
4          38174        QoL              1     0.001174  0.000278  0.009459   

   formula Lipid_Category    Main_Class NumFattyAcylCarbons Unsaturation_Level  
0     22.0    Fatty Acyls  Fatty esters                  10           Very Low  
1     22.0    Fatty Acyls  Fatty esters                  11           Very Low  
2     22.0    Fatty Acyls  Fatty esters                  12           Very Low  
3     22.0    Fatty Acyls  Fatty esters                  12           Very Low  
4     22.0    Fatty Acyls  Fatty esters                  13 

In [79]:
# First, make a copy to avoid modifying original column
merged_metadata_pvalues['Even_Odd_FA'] = 'Unknown'

# Mask rows that are not 'Unknown' and apply conversion
mask_numeric = merged_metadata_pvalues['NumFattyAcylCarbons'] != 'Unknown'

# Convert and classify even vs odd
merged_metadata_pvalues.loc[mask_numeric, 'Even_Odd_FA'] = merged_metadata_pvalues.loc[mask_numeric, 'NumFattyAcylCarbons'].astype(float) % 2
merged_metadata_pvalues['Even_Odd_FA'] = merged_metadata_pvalues['Even_Odd_FA'].replace({0.0: 'Even', 1.0: 'Odd'})


In [80]:
print("Shape of metadata_wide:", metadata_filtered.shape)


Shape of metadata_wide: (1729, 5)


In [81]:
# Create a dictionary to hold each subset of comparisons
comparison_dfs = {
    comp: merged_metadata_pvalues[
        (merged_metadata_pvalues['analysis_group'] == '1') &
        (merged_metadata_pvalues['comparison'] == comp) &
        (~merged_metadata_pvalues['Lipid_Category'].isna())
    ].copy()
    for comp in ['QoL', 'Age', 'Sex', 'BMI']
}


for comp, df in comparison_dfs.items():
    print(f"\n=== {comp} ===")
    print(f"Shape: {df.shape}")
    print(df.head(15))  # You can use df.info() if you want column types too



=== QoL ===
Shape: (1729, 12)
   biomolecule_id comparison analysis_group  effect_size   p_value   q_value  \
0           38167        QoL              1     0.000488  0.220141  0.478704   
1           38168        QoL              1     0.000352  0.154253  0.394402   
2           38172        QoL              1     0.000164  0.607183  0.800352   
3           38173        QoL              1     0.000140  0.663022  0.835220   
4           38174        QoL              1     0.001174  0.000278  0.009459   
5           38176        QoL              1    -0.000300  0.174764  0.422847   
6           38177        QoL              1     0.000590  0.027480  0.151930   
7           38179        QoL              1    -0.000023  0.873283  0.943759   
8           38180        QoL              1     0.000185  0.514724  0.739271   
9           38181        QoL              1     0.000247  0.451115  0.693388   
10          38183        QoL              1    -0.000344  0.163352  0.406526   
11       

In [82]:
#Making sure there are no duplicated values in any of the subsetted data
for comp in ['Age', 'Sex', 'BMI']:
    df = comparison_dfs[comp]
    duplicated = df['biomolecule_id'].duplicated().sum()
    total = df.shape[0]
    unique = df['biomolecule_id'].nunique()

    print(f"\n=== {comp} ===")
    print(f"Total rows: {total}")
    print(f"Unique biomolecule_id: {unique}")
    print(f"Duplicated biomolecule_id entries: {duplicated}")



=== Age ===
Total rows: 1729
Unique biomolecule_id: 1729
Duplicated biomolecule_id entries: 0

=== Sex ===
Total rows: 1729
Unique biomolecule_id: 1729
Duplicated biomolecule_id entries: 0

=== BMI ===
Total rows: 1729
Unique biomolecule_id: 1729
Duplicated biomolecule_id entries: 0


In [83]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gseapy as gp

# Define your inputs
comparisons = ['QoL', 'Age', 'Sex', 'BMI']
metadata_columns = ['Lipid_Category', 'Main_Class', 'Unsaturation_Level', 'Even_Odd_FA']
custom_colors = ["#006D77", "#83C5BE", "#aed9e0", "#FFDDD2", "#E29578", "#444"]

# Initialize summary storage
summary_list = []

# Run GSEA for each comparison
for comp in comparisons:
    print(f"\n=== Performing GSEA for comparison: {comp} ===")
    
    subset_df = merged_metadata_pvalues[
        (merged_metadata_pvalues['analysis_group'] == '1') &
        (merged_metadata_pvalues['comparison'] == comp)
    ].copy()

    if subset_df.empty:
        print(f"No data for comparison '{comp}'. Skipping.")
        continue

    # Ranking
    subset_df['rank'] = np.sign(subset_df['effect_size']) * (-np.log10(subset_df['q_value']))
    subset_df = subset_df[subset_df['rank'] != 0]
    rankings = subset_df.set_index('biomolecule_id')['rank'].sort_values(ascending=False)
    rankings += np.random.uniform(-1e-6, 1e-6, rankings.shape)

    # Create background gene sets
    reference_sets = {}
    for col in metadata_columns:
        for term, group in subset_df.groupby(col):
            if pd.notna(term):
                reference_sets[f"{col}_{term}"] = group['biomolecule_id'].unique().tolist()

    # Run prerank GSEA
    pre_res = gp.prerank(rnk=rankings,
                         gene_sets=reference_sets,
                         threads=4,
                         permutation_num=500000,
                         seed=123,
                         outdir=None)

    if pre_res.res2d.empty:
        print(f"No significant results for '{comp}'. Skipping plotting.")
        continue

    # Sort and clean results
    results_df = pre_res.res2d.sort_values(by='FDR q-val')
    results_df['Comparison'] = comp

    # Replace 0 NOM p-values with minimum possible based on permutations
    min_nom_pval = 1.0 / 10000
    results_df['NOM p-val'] = results_df['NOM p-val'].replace(0.0, min_nom_pval)

    # Save top 20 for the summary table
    summary_terms_df = results_df[results_df['FDR q-val'] < 0.05].sort_values('FDR q-val').head(20)

    summary_list.append(summary_terms_df)

    # Plot only top 6 of those 20
    top_terms_df = summary_terms_df.head(6)
    top_terms = top_terms_df['Term'].tolist()

    if len(top_terms) == 0:
        print(f"No enriched terms with FDR < 0.05 for '{comp}'. Skipping plotting.")
        continue

    hits = [pre_res.results[t]['hits'] for t in top_terms]
    RESs = [pre_res.results[t]['RES'] for t in top_terms]

    # Plot GSEA
    axes = gp.plot.gseaplot2(
        terms=top_terms,
        RESs=RESs,
        hits=hits,
        rank_metric=pre_res.ranking,
        colors=custom_colors[:len(top_terms)],
        legend_kws={'loc': 'center left', 'bbox_to_anchor': (1.2, 0.5), 'frameon': False},
        figsize=(6, 5)
    )

    for ax in axes[:-1]:
        ax.set_xlabel("")
    axes[-1].set_xlabel("Rank", fontsize=14, fontweight='bold')
    plt.suptitle(f"Analysis Group 1 — {comp}", fontsize=16, fontweight='bold')
    plt.tight_layout(rect=[0, 0, 0.8, 0.93])
    plt.savefig(f"GSEA_plot_{comp}.pdf", format='pdf', bbox_inches='tight')
    plt.close()



=== Performing GSEA for comparison: QoL ===


C:\Users\abouelhassan\AppData\Local\Temp\ipykernel_15416\3431601529.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results_df['NOM p-val'] = results_df['NOM p-val'].replace(0.0, min_nom_pval)
C:\Users\abouelhassan\AppData\Local\Temp\ipykernel_15416\3431601529.py:92: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.8, 0.93])



=== Performing GSEA for comparison: Age ===


C:\Users\abouelhassan\AppData\Local\Temp\ipykernel_15416\3431601529.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results_df['NOM p-val'] = results_df['NOM p-val'].replace(0.0, min_nom_pval)
C:\Users\abouelhassan\AppData\Local\Temp\ipykernel_15416\3431601529.py:92: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.8, 0.93])



=== Performing GSEA for comparison: Sex ===


C:\Users\abouelhassan\AppData\Local\Temp\ipykernel_15416\3431601529.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results_df['NOM p-val'] = results_df['NOM p-val'].replace(0.0, min_nom_pval)
C:\Users\abouelhassan\AppData\Local\Temp\ipykernel_15416\3431601529.py:92: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.8, 0.93])



=== Performing GSEA for comparison: BMI ===


C:\Users\abouelhassan\AppData\Local\Temp\ipykernel_15416\3431601529.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results_df['NOM p-val'] = results_df['NOM p-val'].replace(0.0, min_nom_pval)
C:\Users\abouelhassan\AppData\Local\Temp\ipykernel_15416\3431601529.py:92: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.8, 0.93])


In [84]:
#Format p-values
summary_df = pd.concat(summary_list, ignore_index=True)

# Format both p-values in scientific notation with 18 decimal places
summary_df['NOM p-val'] = summary_df['NOM p-val'].apply(lambda x: f"{x:.18e}")
summary_df['FDR q-val'] = summary_df['FDR q-val'].apply(lambda x: f"{x:.18e}")
 
# Save to CSV
summary_df.to_csv("GSEA_enrichment_summary_analysisGroup1_noBatch1_QoL_v2.csv", index=False)
